## Text Mining and NLP

## Part 2

### **Goal**: to internalize the steps, challenges, and methodology of text mining
- explore text analysis by hand
- apply text mining steps in Jupyter with Python libraries NLTK
- classify documents correctly
<br/>
^ This last step will require modeling!

In [106]:
#from __future__ import print_function
import nltk
import sklearn
import pandas as pd
import string, re
import urllib

url_a = "https://raw.githubusercontent.com/aapeebles/text_examples/master/Text%20examples%20folder/A.txt"
url_b = "https://raw.githubusercontent.com/aapeebles/text_examples/master/Text%20examples%20folder/D.txt"
article_a = urllib.request.urlopen(url_a).read()
article_a_st = article_a.decode("utf-8")

In [107]:
# tokens
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
arta_tokens_raw = nltk.regexp_tokenize(article_a_st, pattern)

# lower case
arta_tokens = [i.lower() for i in arta_tokens_raw]

# stop words
nltk.corpus.stopwords.words("english")

stop_words = set(nltk.corpus.stopwords.words('english'))
arta_tokens_stopped = [w for w in arta_tokens if not w in stop_words]

# stem words
stemmer = nltk.SnowballStemmer("english")
arta_stemmed = [stemmer.stem(word) for word in arta_tokens_stopped]

In [108]:
# repeat w second article
article_b = urllib.request.urlopen(url_b).read()
article_b_st = article_b.decode("utf-8")
artb_tokens_raw = nltk.regexp_tokenize(article_b_st, pattern)
artb_tokens = [i.lower() for i in artb_tokens_raw]
artb_tokens_stopped = [w for w in artb_tokens if not w in stop_words]
artb_stemmed = [stemmer.stem(word) for word in artb_tokens_stopped]

### Term Frequency (TF)

$\begin{align}
 tf_{i,j} = \dfrac{n_{i,j}}{\displaystyle \sum_k n_{i,j} }
\end{align} $

### Inverse Document Frequency (IDF)

$\begin{align}
idf(w) = \log \dfrac{N}{df_t}
\end{align} $

### TF-IDF score

$ \begin{align}
w_{i,j} = tf_{i,j} \times \log \dfrac{N}{df_i} \\
tf_{i,j} = \text{number of occurences of } i \text{ in} j \\
df_i = \text{number of documents containing } i \\
N = \text{total number of documents}
\end{align} $


#### Counting from scratch

In [110]:
article_b_st

In [111]:
wordSet = set(arta_stemmed).union(set(artb_stemmed)) 
wordDictA = dict.fromkeys(wordSet, 0) 
wordDictB = dict.fromkeys(wordSet, 0) 

for word in arta_stemmed: 
    wordDictA[word]+=1
    
for word in artb_stemmed: 
    wordDictB[word]+=1    

In [93]:
wordDictA

In [94]:
wordDictB

In [113]:
df = pd.DataFrame([wordDictA, wordDictB], index=range(2))
df.reindex(sorted(df.columns), axis=1)

#### Now with sklearn

In [115]:
cleaned_a = ' '.join(arta_stemmed)
cleaned_b = ' '.join(artb_stemmed)

counts = sklearn.feature_extraction.text.CountVectorizer()
counts_fitted = counts.fit_transform([cleaned_a, cleaned_b])

pd.DataFrame(counts_fitted.toarray(), columns=counts.get_feature_names())

#### Tfidf from scratch

In [116]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count / bowCount
    return tfDict

In [117]:
tfbowA = computeTF(wordDictA, arta_stemmed)
tfbowB = computeTF(wordDictB, artb_stemmed)

In [97]:
tfbowA

In [118]:
def computeIDF(docList):
    """ compute inverse doc freq for each doc in the docList
    returns: IDF for each doc
    """
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / val)
        
    return idfDict

In [119]:
idfs = computeIDF([wordDictA, wordDictB])

In [49]:
idfs

In [120]:
def computeTFIDF(tfBow, idfs):
    """creates function for computing TFIDF"""
    tfidf = {} # creates empty dictionary
    for word, val in tfBow.items(): #starts a for loop using keys (word) and values from tfBow
        tfidf[word] = val * idfs[word] #for each word in tfBow, the value is multiplied by the idfs for the word. 
                                        #The word and resulting computation are then added to the dictionary tfidf
    return tfidf #returns the dictionary tfidf

In [121]:
tfidfBowA = computeTFIDF(tfbowA, idfs)
tfidfBowB = computeTFIDF(tfbowB, idfs)

In [122]:
import pandas as pd
nlpbh = pd.DataFrame([tfidfBowA, tfidfBowB])
nlpbh

Now with sklearn!

In [123]:
# create a string again
cleaned_a = ' '.join(arta_stemmed)
cleaned_b = ' '.join(artb_stemmed)


tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
response = tfidf.fit_transform([cleaned_a, cleaned_b])

nlpskl = pd.DataFrame(response.toarray(), columns=tfidf.get_feature_names())
nlpskl

**Note that these values are different! Sklearn's formula for tf-idf is a little more sophisticated than ours. See the doc for the transformer [here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html).**

### N-grams

Notice that the `TfidfVectorizer()` has a parameter called "ngram_range". Sometimes we want to search not only for individual words but for pairs or triples (etc.) of words. Using $N$ as a variable for the size of the word cluster to consider, we speak of "N-grams". Notice that our default is (1, 1).

### Cosine Similarity

In [101]:
sklearn.metrics.pairwise.cosine_similarity(response)

### Naive Bayes Modeling

Naive Bayes models lend themselves well to NLP problems. Consider the task of trying to predict genre from text. My subjective probability that a text belongs to a certain genre would be a function of the words in the text. So e.g. the (prior) probability that a text is science-fiction may be relatively small. But the probability that a text is science-fiction *given that it uses the word 'cyclotron'* may be quite high.

Now: What's "naive" about Naive Bayes models?

The calculation of the relevant probabilities could get very complicated. But they get much simpler with the (relatively implausible!) assumption that the different features (occurrences of particular words, in our present case of NLP) are *independent*.

In [124]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [102]:
fetch_20newsgroups().target_names

In [125]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [126]:
count = CountVectorizer(stop_words='english')

In [127]:
X_train = count.fit_transform(train.data)
X_test = count.transform(test.data)
y_train = train.target
y_test = test.target

In [103]:
y_test[:10]

Which NB model do you want? Check out the options and their differences [here](https://scikit-learn.org/stable/modules/naive_bayes.html).

In [104]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb.score(X_test, y_test)

In [84]:
from sklearn.metrics import confusion_matrix

In [105]:
confusion_matrix(y_test, nb.predict(X_test))

## Corpus Statistics 

How many non-zero elements are there?

In [140]:
import numpy as np

newval = np.array(df)

non_zero_vals = np.count_nonzero(newval) / float(df.shape[0])
print(f'Average Number of Non-Zero Elements in Vectorized Articles: {non_zero_vals}')

percent_sparse = len([col for col in df.columns if sum(df[col]) <= 1]) / df.shape[1]
print(f'Percentage of columns containing 0: {percent_sparse}')

### Next Steps:
- Create the tf-idf for the **whole** corpus of 12 articles
- What are _on average_ the most important words in the whole corpus?
- Add a column named "Target" to the dataset
- Target will be set to 1 or 0 if the article is "Politics" or "Not Politics"
- Do some exploratory analysis of the dataset
 - What are the average most important words for the "Politics" articles?
 - What are the average most important words for the "Not Politics"?

## Lets talk classification
- How would you split into train and test? what would be the dataset?

In [ ]:
# Sample code
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  